In [1]:
# !pip install -q tabpfn

# jupyter nbconvert --to=html --template=classic *.ipynb --no-input --output=HW1.html

In [2]:
import numpy as np
import pandas as pd
from os.path import exists

repo_url = "https://raw.githubusercontent.com/adrianstando/imbalanced-benchmarking-set/main/datasets/"
dataset = "SpeedDating.csv" # ♥


if exists(dataset):
    # print("File already exists, loading file")
    df = pd.read_csv(dataset, index_col=0)
else:
    try:
        df = pd.read_csv(repo_url + dataset, index_col=0)
        df.to_csv(dataset)
    except:
        print("somthing is not yes")

### SpeedDating

number of rows and columns are:

In [3]:
df.shape

(1048, 19)

Variables:

In [4]:
df.dtypes

pref_o_attractive             float64
pref_o_sincere                float64
pref_o_intelligence           float64
pref_o_funny                  float64
pref_o_ambitious              float64
pref_o_shared_interests       float64
attractive_o                  float64
intelligence_o                float64
funny_o                       float64
shared_interests_o            float64
attractive_important          float64
sincere_important             float64
intellicence_important        float64
funny_important               float64
ambtition_important           float64
shared_interests_important    float64
interests_correlate           float64
like                          float64
TARGET                          int64
dtype: object

Strong class imbalance:

In [5]:
p = df.TARGET.mean()

f"{(1-p)/p:.2f}"  # 4.63 as given in https://github.com/adrianstando/imbalanced-benchmarking-set/blob/main/README.md

'4.63'

### Simple EDA

Densities of variables in both TARGET classes - empirical distributions are similar

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(9, 2, figsize=(12, 22))

axes = axes.ravel()

for i, col in enumerate(df.columns):
    if col != "TARGET":
        sns.histplot(df[df['TARGET'] == 0][col], bins=30, label='TARGET=0', stat='density', kde=True, ax=axes[i])
        sns.histplot(df[df['TARGET'] == 1][col], bins=30, label='TARGET=1', stat='density', kde=True, ax=axes[i])
        axes[i].set_title(f'Histogram for {col} by TARGET')
        axes[i].legend()

for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()        

### 5-cv results

In [7]:
# I use 5 quite different models "rf", "xgb", "lasso",  "knn", "tabpfn"

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_validate #cross_val_score

y = df['TARGET']
X = df.drop(columns=['TARGET'])

# np.random.seed(42) is not needed, as 'random_state' from CV propagetes to models itself
cv = KFold(n_splits=5, shuffle=True, random_state=12)

scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

lasso_logistic = LogisticRegression(penalty='l1', solver='saga', max_iter=5000)

cv_results_lasso = cross_validate(lasso_logistic, X, y, cv=cv, scoring=scoring)
# cv_results_lasso

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=12)

cv_results_rf = cross_validate(rf, X, y, cv=cv, scoring=scoring)
# cv_results_rf

In [10]:
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    max_depth=10,
    learning_rate=0.1,
    n_estimators=100,  
    # early_stopping_rounds=3 # can't use directly in CV setting with train-test split only (val dataset would be needed)
    # scale_pos_weight=df.TARGET.mean()  # using 'scale_pos_weight' this way would be data leakage
)

cv_results_xgb = cross_validate(xgb_clf, X, y, cv=cv, scoring=scoring)
# cv_results_xgb

In [11]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

cv_results_knn = cross_validate(knn, X, y, cv=cv, scoring=scoring)
# cv_results_knn

In [12]:
%%capture
from tabpfn import TabPFNClassifier

tabpfn = TabPFNClassifier(device='cpu', N_ensemble_configurations=10)

cv_results_tabpfn = cross_validate(tabpfn, X, y, cv=cv, scoring=scoring)
# cv_results_tabpfn

# Capture magic is used to not print message to report, message:

# > Loading model that can be used for inference only
# > Using a Transformer with 25.82 M parameters

In [45]:
from typing import Callable

def calc_results_avg(results: dict[str, np.array], agg: Callable[[np.array], float]= np.mean) -> dict[str, float]:
    """aggregate CV results from different folds"""
    return {key: agg(value) for key, value in results.items()}

In [30]:
import pandas as pd

results = pd.DataFrame(
    [
        calc_results_avg(results)
        for results in [
            cv_results_rf,
            cv_results_xgb,            
            cv_results_lasso,
            cv_results_knn,
            cv_results_tabpfn,
        ]
    ]
).T

results.columns = ("rf", "xgb", "lasso",  "knn", "tabpfn")

In [15]:
def highlight_extreme(s:list[float], mode:str='max') -> list[str]:    
    if mode == 'max':
        is_extreme = s == s.max()
    elif mode == 'min':
        is_extreme = s == s.min()
    else:
        return list()

    return ['font-weight: bold' if v else '' for v in is_extreme]

All five models are evaluated using 5-fold cross-validation with identical folds.


* Random Forest is used with default settings, 100 trees are used
* Applied XGboost does not use `early stopping` as it would demand split into three datasets during cross-valiadion, is also does not use `scale_pos_weight` for correcting classes imbalance (applying this yields worsen result)
* Logistic regression with lasso is applied with `C=1`
* `k=5` for kNN is choosen
* TabPFR contains 40 ensembles

Below average of 5 folds from CV.

Dataset is imbalanced so we can disregard accuracy, precision and recall alone. Instead F1 score and ROC AUC should be considered. Random Forest have highest accuracy and precision so also F1 score (which is harmonic mean of both).
Recall is highest for xgboost and ROC AUC (which combines recall and prevalence) for TabPFN.

Overall TabPFN seems to be best modelas F1 is 0.43 for both RF and TabPFN. kNN result are only outlier among 5 models - variables have not been standardized - it could be the reason for that.

In [16]:
# Wyświetlenie DataFrame z wyróżnieniem
results.iloc[2:,:].style.apply(highlight_extreme, axis=1)

Interference time

TabPFN have lowest fit time, but interference is 3 orders of magnitude higher. Lasso (the only linear model used) is fastest.

In [22]:
results.iloc[:2,:].style.apply(highlight_extreme, mode="min", axis=1)

In [33]:
results_std = pd.DataFrame(
    [
        calc_results_avg(results, agg=np.std)
        for results in [
            cv_results_rf,
            cv_results_xgb,            
            cv_results_lasso,
            cv_results_knn,
            cv_results_tabpfn,
        ]
    ]
).T

results_std.columns = ("rf", "xgb", "lasso",  "knn", "tabpfn")

Results of TabPFN are also more stable between folds (standard deviations) then random forest for F1 and ROC AUC:

In [35]:
results_std.loc[["test_f1", "test_roc_auc"]].style.apply(highlight_extreme, mode="min", axis=1)

Detailed TabPFN results for 5-CV:

In [43]:
import numpy as np
for metric, values in cv_results_rf.items():
    if metric[5:] in scoring:
        print(f"{metric:15}: {np.mean(values):.2f} (+/- {np.std(values):.2f})")

test_accuracy  : 0.86 (+/- 0.01)
test_precision : 0.73 (+/- 0.06)
test_recall    : 0.31 (+/- 0.07)
test_f1        : 0.43 (+/- 0.06)
test_roc_auc   : 0.81 (+/- 0.03)


In [ ]:
# conversion to html
jupyter nbconvert --to=html --template=classic *.ipynb --no-input --output=HW1.html